# Установка Automatic1111 и установка зависимостей:
> После успешного запуска и появления ссылки на gradio остановите ячейку и переходите к установке моделей

In [ ]:
#@markdown # Для установки зависимостей, после успешного старта можно остановить и подгрузить модели
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets
import os
import time

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

clear_output()
print("[0;33mУстанавливаем AUTOMATIC1111...")
#python -c "import sys; print('\n'.join(sys.path))" - этим в терминале можно проверить path в kaggle
#/opt/conda/lib/python3.10/site-packages - актуальная версия окружения

with capture.capture_output() as cap:
    if not os.path.exists('/kaggle/working/drive'):
        !mkdir -p /kaggle/working/drive
    %cd /kaggle/working/drive
    fgitclone = "git clone --depth 1"
    if not os.path.exists('/kaggle/working/drive/sd'):
        !mkdir -p /kaggle/working/drive/sd
    %cd /kaggle/working/drive/sd
    !git clone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
    !mkdir -p /kaggle/working/drive/sd/stable-diffusion-webui/cache/
    os.environ['TRANSFORMERS_CACHE']=f"/kaggle/working/drive/sd/stable-diffusion-webui/cache"

with capture.capture_output() as cap:
  %cd /kaggle/working/drive/sd/stable-diffusion-webui/
  time.sleep(1)
  !rm webui.sh
  !git pull
    
#%cd /kaggle/working/drive/sd/stable-diffusion-webui    
#!python3 launch.py --share --enable-insecure-extension-access --theme dark --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars
clear_output()
print("[0;33mAUTOMATIC1111 установлен")
inf('\u2714 Успешно!','success', '50px')

# Скачивание Моделей с huggingface/civitai:
> В поле URL введите ссылку на модель и запустите ячейку. Будет установлена нужная модель и VAE

In [ ]:
import os
import requests
from tqdm.auto import tqdm
###################################################
#@markdown # Скачивание Моделей с huggingface/civitai
# Тут укажите ссылку на модель
URL = "https://civitai.com/api/download/models/15236?type=Model&format=SafeTensor&size=full&fp=fp16" #@param {type:"string"} 
# Тут укажите тип загружаемой модели, варианты перечислены в скобках []
Directory = "Model" #@param ["Model", "VAE", "Lora", "Hypernetworks", "Textual Inversions"]
#@markdown - Если нужно в другое место укажи путь.
path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/" #@param {type:"string"}
VAE_predownload = "vae-ft-mse-840000-ema-pruned.safetensors" #@param ["none", "vae-ft-mse-840000-ema-pruned.safetensors"]

###################################################



if path:
  destination_path = path
else:
  if Directory == "Model":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/"
  elif Directory == "VAE":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/"
  elif Directory == "Lora":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/Lora/"
  elif Directory == "Hypernetworks":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/models/hypernetworks/"
  elif Directory == "Textual Inversions":
    destination_path = "/kaggle/working/drive/sd/stable-diffusion-webui/embeddings/"

if VAE_predownload == "vae-ft-mse-840000-ema-pruned.safetensors":
  if os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors'):
      inf('\u2714 Done VAE уже есть в папке','success', '50px')
  else:
      inf('\u2718 Загружаем VAE с https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors','success', "250px")
      %cd /kaggle/working/
      !wget -O /kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
      inf('\u2714 VAE успешно загружено!','success', '200px')
    
if not URL:
    print("Вы не ввели ссылку для скачивания.")
else:
    if "civitai" in URL:
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024 # 1 Kibibyte
        progress_bar_length = 512
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True, ncols=progress_bar_length)
        filename = response.headers.get('content-disposition')
        if filename:
            filename = filename.split('filename=')[1].strip('\"')
        else:
            filename = URL.split("/")[-1]
        destination_file_path = os.path.join(destination_path, filename)

        # Create the directory if it does not exist
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        with open(destination_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    progress_bar.update(len(chunk))
        progress_bar.close()

    elif "huggingface" in URL:
        response = requests.get(URL, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024 # 1 Kibibyte
        progress_bar_length = 512
        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True, ncols=progress_bar_length)

        filename = URL.split("/")[-1]
        destination_file_path = os.path.join(destination_path, filename)

        # Create the directory if it does not exist
        if not os.path.exists(destination_path):
            os.makedirs(destination_path)

        with open(destination_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    progress_bar.update(len(chunk))
        progress_bar.close()

    if path:
        print(f"Файл {filename} загрузился в папку для {path}.")
    else:
        print(f"Файл {filename} загрузился в папку для {Directory}.")

# Установка моделей ControlNet во временную папку 
> Подгрузим сразу все модели ControlNet в виртуальную машину на Kaggle используя символическую ссылку на рабочую папку SD

In [ ]:
#@markdown # TEMP ControlNet - Загрузка моделей во временную папку

from IPython.display import clear_output
import os

%cd /tmp
if not os.path.exists('/tmp/ControlNet'):
    !sudo mkdir /tmp/ControlNet/
!sudo wget -O /tmp/ControlNet/control_v11e_sd15_ip2p.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth
!sudo wget -O /tmp/ControlNet/control_v11e_sd15_ip2p.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.yaml
!sudo wget -O /tmp/ControlNet/control_v11e_sd15_shuffle.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth
!sudo wget -O /tmp/ControlNet/control_v11e_sd15_shuffle.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.yaml
!sudo wget -O /tmp/ControlNet/control_v11f1e_sd15_tile.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth
!sudo wget -O /tmp/ControlNet/control_v11f1e_sd15_tile.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.yaml
!sudo wget -O /tmp/ControlNet/control_v11f1p_sd15_depth.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth
!sudo wget -O /tmp/ControlNet/control_v11f1p_sd15_depth.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_canny.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_canny.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_inpaint.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_inpaint.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_lineart.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_lineart.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_mlsd.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_mlsd.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_normalbae.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_normalbae.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_openpose.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_openpose.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_scribble.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_scribble.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_seg.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_seg.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_softedge.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15_softedge.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.yaml
!sudo wget -O /tmp/ControlNet/control_v11p_sd15s2_lineart_anime.pth https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth
!sudo wget -O /tmp/ControlNet/control_v11p_sd15s2_lineart_anime.yaml https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.yaml
!sudo wget -O /tmp/ControlNet/t2iadapter_style-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/blob/main/t2iadapter_style-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_sketch-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_seg-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_openpose-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_openpose-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_keypose-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_depth-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_depth-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_color-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_color-fp16.safetensors
!sudo wget -O /tmp/ControlNet/t2iadapter_canny-fp16.safetensors https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_canny-fp16.safetensors
clear_output()
if not os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet'):    
    !mkdir /kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet
if os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet/tmp'):
    !rm -r /kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet/tmp
!mkdir /kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet/tmp
%cd /kaggle/working/drive/sd/stable-diffusion-webui/models/
!sudo ln -s /tmp/ControlNet /kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet/tmp
clear_output()
!ls /kaggle/working/drive/sd/stable-diffusion-webui/models/ControlNet/
inf('\u2714 Ссылка на /tmp/ControlNet создана!','success', '350px')

# Установка моделей SD во временную папку

In [ ]:
#@markdown # TEMP Models - Загрузка моделей во временную папку

import time
import sys
import fileinput

VAE_predownload = "vae-ft-mse-840000-ema-pruned.safetensors" #@param ["none", "vae-ft-mse-840000-ema-pruned.safetensors"]

Deliberate_v2 = True #@param {type:"boolean"}
Deliberate2_inpainting = True #@param {type:"boolean"}
Lyriel_v15 = False #@param {type:"boolean"}
ProjectAIO_v50 = False #@param {type:"boolean"}
epicrealism_newAge = False #@param {type:"boolean"}
theallysMixIV_v10 = False #@param {type:"boolean"}
cyberrealistic_v20 = False #@param {type:"boolean"}
fantasticmix_v40 = False #@param {type:"boolean"}

%cd /tmp
if not os.path.exists('/tmp/TempModels'):
    !sudo mkdir /tmp/TempModels/
if Deliberate_v2:
    !sudo wget -O /tmp/TempModels/Deliberate_v2.safetensors https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate_v2.safetensors
if Deliberate2_inpainting:
    !sudo wget -O /tmp/TempModels/Deliberate-inpainting.safetensors https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate-inpainting.safetensors
if Lyriel_v15:
    !sudo wget -O /tmp/TempModels/lyriel_v15.safetensors 'https://civitai.com/api/download/models/50127?type=Model&format=SafeTensor&size=full&fp=fp16'
if ProjectAIO_v50:
    !sudo wget -O /tmp/TempModels/projectAIO_v50.safetensors 'https://civitai.com/api/download/models/59746?type=Model&format=SafeTensor&size=full&fp=fp16'
if epicrealism_newAge:
    !sudo wget -O /tmp/TempModels/epicrealism_newAge.safetensors 'https://civitai.com/api/download/models/48174?type=Model&format=SafeTensor&size=full&fp=fp16'
if theallysMixIV_v10:
    !sudo wget -O /tmp/TempModels/theallysMixIV_v10.safetensors 'https://civitai.com/api/download/models/45669?type=Model&format=SafeTensor&size=pruned&fp=fp16'
if cyberrealistic_v20:
    !sudo wget -O /tmp/TempModels/cyberrealistic_v20.safetensors 'https://civitai.com/api/download/models/55015?type=Model&format=SafeTensor&size=full&fp=fp32'
if fantasticmix_v40:
    !sudo wget -O /tmp/TempModels/fantasticmix_v40.safetensors 'https://civitai.com/api/download/models/59385?type=Model&format=SafeTensor&size=pruned&fp=fp16'

if VAE_predownload == "vae-ft-mse-840000-ema-pruned.safetensors":
    if os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors'):
        print("[0;33mDone VAE уже есть в папке!")
    else:
        print("[0;33mЗагружаем VAE с https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")
        %cd /kaggle/working/
        !wget -O /kaggle/working/drive/sd/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
        clear_output()
        print("[0;33mVAE успешно загружено!")
if not os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion'):    
    !mkdir /kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion
if os.path.exists('/kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/tmp'):
    !rm -r /kaggle/working/drive/sd/stable-diffusion-webui/models/tmp
!mkdir /kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/tmp
!sudo ln -s /tmp/TempModels /kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/tmp
clear_output()
!ls /kaggle/working/drive/sd/stable-diffusion-webui/models/Stable-diffusion/
print("[0;33mМодели загружены")
inf('\u2714 Ссылка на /tmp/TempModels создана!','success', '350px')  

# Подключаем яндекс.диск - Смотрите в консоль:
> Предложит войти на ya.ru/device и ввести код из консоли для разрешения доступа к диску! 
> Внимание! Будет синхронизовано всё содержимое яндекс.диска. 
> Используйте чистый акк!

In [ ]:
#@markdown # Смотрите в консоль, она предложит войти на ya.ru/device и ввести код из консоли для разрешения доступа к диску! Внимание! Будет синхронизовано всё содержимое яндекс.диска. Используйте чистый акк!
from IPython.display import clear_output
import ipywidgets as widgets
import os


def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

print("[0;33mУстанавливаем клиент Яндекс.Диска...")
!sudo wget https://repo.yandex.ru/yandex-disk/yandex-disk_latest_amd64.deb -O /tmp/yandex-disk.deb && sudo dpkg -i /tmp/yandex-disk.deb
!sudo apt-get install yandex-disk
clear_output()
print("[0;33mАвторизуем клиент Яндекс.Диска...")
!yandex-disk token
if not os.path.exists('/kaggle/working/drive'):
    !mkdir -p /kaggle/working/drive
!yandex-disk stop --dir=/kaggle/working/drive
clear_output()
print("[0;33mЗапускаем клиент Яндекс.Диска...")
!yandex-disk start --dir=/kaggle/working/drive
clear_output()
print("[0;33mКлиент Яндекс.Диска запущен")
inf('\u2714 Успешно!','success', '50px')

# Запуск Stable diffusion: 

In [ ]:
from IPython.display import clear_output
import gc


##########################################
#Сюда вставить ↓ свой токен от Ngrok
Ngrok_token = "" #@param {type:"string"}
##########################################


vr_nt=''
if Ngrok_token!="":
    vr_nt=f"--ngrok {Ngrok_token}"
else:
    vr_nt=''

%cd /kaggle/working/drive/sd/stable-diffusion-webui
clear_output()
print("\033[0mЗапускаем stable-diffusion-webui...")
gc.collect()

!python launch.py --share $vr_nt --theme dark --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars
